<a href="https://colab.research.google.com/github/gachet/ad-1-24/blob/main/pca/ejem-pres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np # importando numpy
from scipy import stats # importando scipy.stats
import pandas as pd # importando pandas



In [ ]:
x = [19,39,30,30,15,15,15,30]
y= [63,74,87,23,35,43,32,73]
datos = np.stack((x, y), axis=0)
datos

array([[19, 39, 30, 30, 15, 15, 15, 30],
       [63, 74, 87, 23, 35, 43, 32, 73]])

In [ ]:
A=np.cov(datos)

In [ ]:
autovalor, autovector = np.linalg.eig(A)

In [ ]:
autovalor

array([ 56.10230167, 580.80841262])

In [ ]:
autovector

array([[-0.97124992, -0.23806218],
       [ 0.23806218, -0.97124992]])

In [3]:
x = [2.5,0.5,2.2,1.9,3.1,2.3,2,1,1.5,1.2,1.8]
y= [2.4,0.7,2.9,2.2,3.0,2.7,1.6,1.1,1.6,0.9,1.91]
datos = np.stack((x, y), axis=0)
datos

array([[2.5 , 0.5 , 2.2 , 1.9 , 3.1 , 2.3 , 2.  , 1.  , 1.5 , 1.2 , 1.8 ],
       [2.4 , 0.7 , 2.9 , 2.2 , 3.  , 2.7 , 1.6 , 1.1 , 1.6 , 0.9 , 1.91]])

In [5]:
A=np.cov(datos)
A

array([[0.54163636, 0.5438    ],
       [0.5438    , 0.6449    ]])

In [6]:
autovalor, autovector = np.linalg.eig(A)

In [7]:
autovalor

array([0.04702256, 1.13951381])

In [8]:
autovector

array([[-0.73977066, -0.67285911],
       [ 0.67285911, -0.73977066]])